# Interactive Estimator
This estimator is designed to be an interactive calculator, where the user can input a set of parameters that match their situation, and see the comparisons between buying, building and renting.

## Layout 
- Costs
  - We will assume that utilities are the same between the three types
  - Buying costs
    - Down payment
    - Closing costs
      - 2-5% of principle
    - Property taxes
    - Homeowners insurance
    - HOA fees
    - Maintenance
      - $1 per sq. ft. or 1%
  - Building costs
    - Land
    - Lot prep
    - Construction
    - Homeowners insurance
    - HOA fees
    - Maintenance
      - Less than buying an existing house, for a time
  - Renting costs
    - Rent
    - Renters insurance
- Final investment value
  - Buying
    - The value of the home after appreciation, minus the total cumulative costs
    - Selling and buying while moving will also need to be considered
  - Building
    - Same as buying, except building subsequent homes instead of purchasing a home
  - Renting
    - Value of the investment of the down payment and any periods in which the cost of buying or building is greater and the renter chooses to invest the difference



In [1]:
# All import statements
import ipywidgets as widgets
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from math import e
from IPython.display import display
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import DateOffset

# Global options
pd.set_option('display.max_columns', None)

In [2]:
style = {'description_width': 'initial'}

In [3]:
# Purchase parameters
purch_cost = widgets.BoundedIntText(value=600000, min=0, max=10000000, description='Purchase Cost', disabled=False, style=style)
purch_close_cost = widgets.BoundedFloatText(value=0.050, min=0, max=1, step=0.001, description='Closing Cost Percentage', disabled=False, style=style)
purch_loan_years = widgets.BoundedIntText(value=30, min=0, max=10000000, description='Loan Years', disabled=False, style=style)
purch_initi_hoa = widgets.BoundedIntText(value=150, min=0, max=10000000, description='Initial HOA per Month', disabled=False, style=style)
purch_apprec = widgets.BoundedFloatText(value=0.057, min=0, max=1, step=0.001, description='Home Appreciation Rate', disabled=False, style=style)
purch_loan_rate = widgets.BoundedFloatText(value=0.040, min=0, max=1, step=0.001, description='Home Loan Interest Rate', disabled=False, style=style)

# Build parameters
property_cost = widgets.BoundedIntText(value=200000, min=0, max=10000000, description='Property Cost', disabled=False, style=style)
const_cost = widgets.BoundedIntText(value=300000, min=0, max=10000000, description='Construction Cost', disabled=False, style=style)
built_value = widgets.BoundedIntText(value=600000, min=0, max=10000000, description='Built Value', disabled=False, style=style)
const_close_cost = widgets.BoundedFloatText(value=0.050, min=0, max=1, step=0.001, description='Closing Cost Percentage', disabled=False, style=style)
const_loan_years = widgets.BoundedIntText(value=30, min=0, max=10000000, description='Loan Years', disabled=False, style=style)
const_mos = widgets.BoundedIntText(value=12, min=0, max=10000000, description='Months of Construction', disabled=False, style=style)
const_init_hoa = widgets.BoundedIntText(value=150, min=0, max=10000000, description='Initial HOA per Month', disabled=False, style=style)
const_apprec = widgets.BoundedFloatText(value=0.057, min=0, max=1, step=0.001, description='Home Appreciation Rate', disabled=False, style=style)
const_loan_rate = widgets.BoundedFloatText(value=0.05, min=0, max=1, step=0.001, description='Construction Loan Interest Rate', disabled=False, style=style)

# Purchase and build shared parameters
down_payment = widgets.BoundedIntText(value=80000, min=0, max=10000000, description='Down Payment', disabled=False, style=style)
maint_per_sqft = widgets.BoundedFloatText(value=0.750, min=0, max=1, step=0.001, description='Maintenance Cost per Sq. Ft.', disabled=False, style=style)
assess_rate = widgets.BoundedFloatText(value=0.070, min=0, max=1, step=0.001, description='Tax Assess Rate', disabled=False, style=style)
mill_levy = widgets.BoundedIntText(value=95, min=0, max=10000000, description='Tax Mill Levy', disabled=False, style=style)
square_footage = widgets.BoundedIntText(value=1750, min=0, max=10000000, description='Square Footage', disabled=False, style=style)

# Rent parameters
initial_rent = widgets.BoundedIntText(value=2400, min=0, max=10000000, description='Initial Monthly Rent', disabled=False, style=style)
rent_growth_rate = widgets.BoundedFloatText(value=0.035, min=0, max=1, step=0.001, description='Rent Growth Rate', disabled=False, style=style)
invest_rate = widgets.BoundedFloatText(value=0.0992, min=0, max=1, step=0.0001, description='Market Growth Rate', disabled=False, style=style)

# General parameters
years_to_consider = widgets.BoundedIntText(value=45, min=0, max=10000000, description='Years to Consider', disabled=False, style=style)
inflation = widgets.BoundedFloatText(value=0.02, min=0, max=1, step=0.001, description='Rate of Inflation', disabled=False, style=style)

In [4]:
# Parameter chunks
text1 = 'Purchase Parameters'
text2 = 'Building Parameters'
text3 = 'Purchase and Build Shared Parameters'
text4 = 'Rent Parameters'
text5 = 'General Parameters'

purchase_params = widgets.VBox(
    [widgets.HTML(value = f"<b><font color='red'>{text1}</b>"),
    purch_cost,
    purch_close_cost,
    purch_loan_years,
    purch_initi_hoa,
    purch_apprec,
    purch_loan_rate]
)
build_params = widgets.VBox(
    [widgets.HTML(value = f"<b><font color='red'>{text2}</b>"),
    property_cost,
    const_cost,
    built_value,
    const_close_cost,
    const_loan_years,
    const_mos,
    const_init_hoa,
    const_apprec,
    const_loan_rate]
)
purchase_build_shared_params = widgets.VBox(
    [widgets.HTML(value = f"<b><font color='red'>{text3}</b>"),
    down_payment,
    maint_per_sqft,
    assess_rate,
    mill_levy,
    square_footage]
)
rent_params = widgets.VBox(
    [widgets.HTML(value = f"<b><font color='red'>{text4}</b>"),
    initial_rent,
    rent_growth_rate,
    invest_rate]
)
gen_params = widgets.VBox(
    [widgets.HTML(value = f"<b><font color='red'>{text5}</b>"),
    years_to_consider,
    inflation]
)

purch_cost, purch_close_cost, purch_loan_years, purch_initi_hoa, purch_apprec, purch_loan_rate, property_cost, const_cost, 
built_value, const_close_cost, const_loan_years, const_mos, const_init_hoa, const_apprec, const_loan_rate, down_payment,
maint_per_sqft, assess_rate, mill_levy, square_footage, initial_rent, rent_growth_rate, invest_rate, years_to_consider, inflation

In [5]:
def comparison(purch_cost, purch_close_cost, purch_loan_years, purch_initi_hoa, purch_apprec, purch_loan_rate, property_cost, const_cost, 
built_value, const_close_cost, const_loan_years, const_mos, const_init_hoa, const_apprec, const_loan_rate, down_payment,
maint_per_sqft, assess_rate, mill_levy, square_footage, initial_rent, rent_growth_rate, invest_rate, years_to_consider, inflation):

    # Conversions
    tot_const_loan_amt = property_cost + const_cost - down_payment
    const_loan_rate_mon = const_loan_rate / 12
    invest_rate_mon = invest_rate / 12
    rent_growth_rate_monthly = rent_growth_rate / 12
    const_apprec_mon = const_apprec / 12
    current_month = datetime.date.today().replace(day=1)
    const_loan_mos = const_loan_years * 12
    months_to_consider = years_to_consider * 12
    inflation_mon = inflation / 12
    purch_loan_mos = purch_loan_years * 12
    purch_loan_rate_mon = purch_loan_rate / 12
    purch_apprec_mon = purch_apprec / 12
    tot_purch_loan_amt = purch_cost - down_payment

    # Create blank table for monthly level data
    monthly_data = pd.DataFrame()

    # Create Period to offset the index by one, to indicate month and create periods
    monthly_data['Overall Month'] = range(1, months_to_consider + 2)
    monthly_data['Overall Year'] = ((monthly_data['Overall Month'] - 1) // 12)
    monthly_data['Year Start'] = np.where((((monthly_data['Overall Month'] - 1) // 12) == (monthly_data['Overall Month'] - 1) / 12) | (monthly_data['Overall Month'] == 1), 1, 0)
    monthly_data['Construction Month'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Overall Month'], 0)
    monthly_data['Construction Loan Month'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), monthly_data['Overall Month'] - const_mos, 0)
    monthly_data['Purchase Loan Month'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, monthly_data['Overall Month'], 0)

    # Calculate home value
    monthly_data['Build Home Value'] = np.where(monthly_data['Overall Month'] <= const_mos, 0, built_value * (e**(const_apprec_mon * (monthly_data['Overall Month'] - const_mos))))

    monthly_data['Purchase Home Value'] = purch_cost * (e**(purch_apprec_mon * (monthly_data['Overall Month'])))

    # Calculate loan value
    monthly_data['Construction Loan Value'] = np.where(monthly_data['Construction Loan Month'] != 0, tot_const_loan_amt * (((1 + const_loan_rate_mon) ** (const_loan_mos)) - ((1 + const_loan_rate_mon)**(monthly_data['Construction Loan Month'] - 1))) / ((1 + const_loan_rate_mon)**(const_loan_mos) - 1), tot_const_loan_amt)
    monthly_data['Construction Loan Value'] = np.where(monthly_data['Overall Month'] > (const_mos + const_loan_mos), 0, monthly_data['Construction Loan Value'])

    monthly_data['Purchase Loan Value'] = np.where(monthly_data['Purchase Loan Month'] != 0, tot_purch_loan_amt * (((1 + purch_loan_rate_mon) ** (purch_loan_mos)) - ((1 + purch_loan_rate_mon)**(monthly_data['Purchase Loan Month'] - 1))) / ((1 + purch_loan_rate_mon)**(purch_loan_mos) - 1), tot_purch_loan_amt)
    monthly_data['Purchase Loan Value'] = np.where(monthly_data['Overall Month'] > purch_loan_mos, 0, monthly_data['Purchase Loan Value'])

    # Calculate disbursement payments in a linear fashion
    monthly_data['Construction Disbursements'] = np.where(monthly_data['Overall Month'] <= const_mos, tot_const_loan_amt / const_mos, 0)

    # Calculate monthly payments
    monthly_data['Construction Mortgage Payment'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), tot_const_loan_amt * (const_loan_rate_mon * ((1 + const_loan_rate_mon)**const_loan_mos)) / ((1 + const_loan_rate_mon)**const_loan_mos - 1), 0)

    monthly_data['Purchase Mortgage Payment'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, tot_purch_loan_amt * (purch_loan_rate_mon * ((1 + purch_loan_rate_mon)**purch_loan_mos)) / ((1 + purch_loan_rate_mon)**purch_loan_mos - 1), 0)

    # Calculate interest payment
    monthly_data['Construction Interest Payment'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Construction Disbursements'].cumsum() * const_loan_rate_mon, monthly_data['Construction Loan Value'] * const_loan_rate_mon)
    monthly_data['Construction Cumulative Interest Payments'] = monthly_data['Construction Interest Payment'].cumsum()

    monthly_data['Purchase Interest Payment'] = monthly_data['Purchase Loan Value'] * purch_loan_rate_mon
    monthly_data['Purchase Cumulative Interest Payments'] = monthly_data['Purchase Interest Payment'].cumsum()

    # Calculate principal payments
    monthly_data['Construction Principal Payment'] = np.where(monthly_data['Overall Month'] > const_mos, monthly_data['Construction Mortgage Payment'] - monthly_data['Construction Interest Payment'], 0)
    monthly_data['Construction Cumulative Principal Payments'] = monthly_data['Construction Principal Payment'].cumsum()

    monthly_data['Purchase Principal Payment'] = monthly_data['Purchase Mortgage Payment'] - monthly_data['Purchase Interest Payment']
    monthly_data['Purchase Cumulative Principal Payments'] = monthly_data['Purchase Principal Payment'].cumsum()

    # Calculate property taxes
    monthly_data['Construction Property Tax Payment'] = np.where(monthly_data['Overall Month'] > const_mos, (((monthly_data['Build Home Value'] * assess_rate) * (mill_levy / 1000)) / 12), 0)
    monthly_data['Construction Cumulative Property Taxes'] = monthly_data['Construction Property Tax Payment'].cumsum()

    monthly_data['Purchase Property Tax Payment'] = (((monthly_data['Purchase Home Value'] * assess_rate) * (mill_levy / 1000)) / 12)
    monthly_data['Purchase Cumulative Property Taxes'] = monthly_data['Purchase Property Tax Payment'].cumsum()

    # Calculate maintenance costs
    monthly_data['Construction Maintenance Costs'] = np.where(monthly_data['Overall Month'] > const_mos, ((square_footage * maint_per_sqft) / 12), 0)
    monthly_data['Construction Cumulative Maintenance Costs'] = monthly_data['Construction Maintenance Costs'].cumsum()

    monthly_data['Purchase Maintenance Costs'] = ((square_footage * maint_per_sqft) / 12)
    monthly_data['Purchase Cumulative Maintenance Costs'] = monthly_data['Purchase Maintenance Costs'].cumsum()

    # Calculate HOA costs
    monthly_data['Construction HOA Costs'] = const_init_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
    monthly_data['Construction Cumulative HOA Costs'] = monthly_data['Construction HOA Costs'].cumsum()

    monthly_data['Purchase HOA Costs'] = purch_initi_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
    monthly_data['Purchase Cumulative HOA Costs'] = monthly_data['Purchase HOA Costs'].cumsum()

    # Calculate total home payments
    monthly_data['Construction Total Home Payments'] = monthly_data['Construction Interest Payment'] + monthly_data['Construction Principal Payment'] + monthly_data['Construction Property Tax Payment'] + monthly_data['Construction Maintenance Costs'] + monthly_data['Construction HOA Costs']
    monthly_data['Construction Cumulative Home Payments'] = monthly_data['Construction Total Home Payments'].cumsum() + down_payment + (const_close_cost * tot_const_loan_amt)

    monthly_data['Purchase Total Home Payments'] = monthly_data['Purchase Interest Payment'] + monthly_data['Purchase Principal Payment'] + monthly_data['Purchase Property Tax Payment'] + monthly_data['Purchase Maintenance Costs'] + monthly_data['Purchase HOA Costs']
    monthly_data['Purchase Cumulative Home Payments'] = monthly_data['Purchase Total Home Payments'].cumsum() + down_payment + (purch_close_cost * tot_purch_loan_amt)

    # Calculate equity metrics
    monthly_data['Construction Equity Ownership Proportion'] = 1- (monthly_data['Construction Loan Value'] / tot_const_loan_amt)
    monthly_data['Construction Equity Value'] = monthly_data['Construction Equity Ownership Proportion'] * monthly_data['Build Home Value']
    monthly_data['Construction Equity Gain'] = monthly_data['Construction Equity Value'] - monthly_data['Construction Cumulative Home Payments']
    monthly_data['Construction Gains upon Sale'] = monthly_data['Build Home Value'] - monthly_data['Construction Cumulative Home Payments'] - monthly_data['Construction Loan Value']

    monthly_data['Purchase Equity Ownership Proportion'] = 1- (monthly_data['Purchase Loan Value'] / tot_purch_loan_amt)
    monthly_data['Purchase Equity Value'] = monthly_data['Purchase Equity Ownership Proportion'] * monthly_data['Purchase Home Value']
    monthly_data['Purchase Equity Gain'] = monthly_data['Purchase Equity Value'] - monthly_data['Purchase Cumulative Home Payments']
    monthly_data['Purchase Gains upon Sale'] = monthly_data['Purchase Home Value'] - monthly_data['Purchase Cumulative Home Payments'] - monthly_data['Purchase Loan Value']

    # Calculate rent payments
    monthly_data['Rent Payments'] = np.where(monthly_data['Overall Month'] == 1, (initial_rent + down_payment), initial_rent * (e**(rent_growth_rate_monthly * (monthly_data['Overall Month'] - 1))))
    monthly_data['Cumulative Rent Payments'] = monthly_data['Rent Payments'].cumsum()

    # Calculate rent investment value
    monthly_data['Rent Investment'] = down_payment * (e**(invest_rate_mon * (monthly_data['Overall Month'] - 1)))

    # Shrink set to year starts for simpler graphs
    year_data = monthly_data.loc[monthly_data['Year Start'] == 1]

    # Plot payment comparison
    plt.plot(year_data['Overall Year'], year_data['Rent Investment'], color='red', label='Renting')
    plt.plot(year_data['Overall Year'], year_data['Construction Equity Gain'], color='blue', label='Construction Equity Gain')
    plt.plot(year_data['Overall Year'], year_data['Construction Gains upon Sale'], color='purple', label='Construction Gains upon Sale')
    plt.plot(year_data['Overall Year'], year_data['Purchase Equity Gain'], color='green', label='Purchase Equity Gain')
    plt.plot(year_data['Overall Year'], year_data['Purchase Gains upon Sale'], color='grey', label='Purchase Gains upon Sale')
    plt.figlegend(loc = 'upper left')
    plt.ticklabel_format(style='plain', useLocale=0)
    plt.figure(figsize=(10,10))
    display(plt)
    return plt

In [6]:
outs = widgets.interactive_output(comparison, {'purch_cost':purch_cost, 'purch_close_cost':purch_close_cost, 'purch_loan_years':purch_loan_years, 'purch_initi_hoa':purch_initi_hoa, 'purch_apprec':purch_apprec, 'purch_loan_rate':purch_loan_rate, 'property_cost':property_cost, 'const_cost':const_cost, 'built_value':built_value, 'const_close_cost':const_close_cost, 'const_loan_years':const_loan_years, 'const_mos':const_mos, 'const_init_hoa':const_init_hoa, 'const_apprec':const_apprec, 'const_loan_rate':const_loan_rate, 'down_payment':down_payment, 'maint_per_sqft':maint_per_sqft, 'assess_rate':assess_rate, 'mill_levy':mill_levy, 'square_footage':square_footage, 'initial_rent':initial_rent, 'rent_growth_rate':rent_growth_rate, 'invest_rate':invest_rate, 'years_to_consider':years_to_consider, 'inflation':inflation})
outs.layout.width = '100%'

In [7]:
grid_layout = widgets.GridspecLayout(3,3)
grid_layout[0,0] = widgets.VBox([purchase_params,rent_params])
grid_layout[0,1] = build_params
grid_layout[0,2] = widgets.VBox([purchase_build_shared_params,gen_params,widgets.Button(description='Calculate', disabled=False, icon='check')])
grid_layout[1:2,0:2] = outs

grid_layout

GridspecLayout(children=(VBox(children=(VBox(children=(HTML(value="<b><font color='red'>Purchase Parameters</b…